# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

from dotenv import load_dotenv
%load_ext dotenv
%dotenv 

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.

import os
PRICE_DATA = os.getenv("PRICE_DATA")
from glob import glob
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
print("Parquet files found:", parquet_files)
parquet_files

Parquet files found: ['../../05_src/data/prices\\A\\A_2013.parquet', '../../05_src/data/prices\\A\\A_2014.parquet', '../../05_src/data/prices\\A\\A_2015.parquet', '../../05_src/data/prices\\A\\A_2016.parquet', '../../05_src/data/prices\\A\\A_2017.parquet', '../../05_src/data/prices\\A\\A_2018.parquet', '../../05_src/data/prices\\A\\A_2019.parquet', '../../05_src/data/prices\\A\\A_2020.parquet', '../../05_src/data/prices\\A\\A_2021.parquet', '../../05_src/data/prices\\A\\A_2022.parquet', '../../05_src/data/prices\\A\\A_2023.parquet', '../../05_src/data/prices\\A\\A_2024.parquet', '../../05_src/data/prices\\ABNB\\ABNB_2020.parquet', '../../05_src/data/prices\\ABNB\\ABNB_2021.parquet', '../../05_src/data/prices\\ABNB\\ABNB_2022.parquet', '../../05_src/data/prices\\ABNB\\ABNB_2023.parquet', '../../05_src/data/prices\\ABNB\\ABNB_2024.parquet', '../../05_src/data/prices\\ABT\\ABT_2013.parquet', '../../05_src/data/prices\\ABT\\ABT_2014.parquet', '../../05_src/data/prices\\ABT\\ABT_2015.parque

['../../05_src/data/prices\\A\\A_2013.parquet',
 '../../05_src/data/prices\\A\\A_2014.parquet',
 '../../05_src/data/prices\\A\\A_2015.parquet',
 '../../05_src/data/prices\\A\\A_2016.parquet',
 '../../05_src/data/prices\\A\\A_2017.parquet',
 '../../05_src/data/prices\\A\\A_2018.parquet',
 '../../05_src/data/prices\\A\\A_2019.parquet',
 '../../05_src/data/prices\\A\\A_2020.parquet',
 '../../05_src/data/prices\\A\\A_2021.parquet',
 '../../05_src/data/prices\\A\\A_2022.parquet',
 '../../05_src/data/prices\\A\\A_2023.parquet',
 '../../05_src/data/prices\\A\\A_2024.parquet',
 '../../05_src/data/prices\\ABNB\\ABNB_2020.parquet',
 '../../05_src/data/prices\\ABNB\\ABNB_2021.parquet',
 '../../05_src/data/prices\\ABNB\\ABNB_2022.parquet',
 '../../05_src/data/prices\\ABNB\\ABNB_2023.parquet',
 '../../05_src/data/prices\\ABNB\\ABNB_2024.parquet',
 '../../05_src/data/prices\\ABT\\ABT_2013.parquet',
 '../../05_src/data/prices\\ABT\\ABT_2014.parquet',
 '../../05_src/data/prices\\ABT\\ABT_2015.parquet'

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.
dd_px = dd.read_parquet(parquet_files)
dd_px 

#dd_px =dd_px.compute()
# Load parquet files into a Dask DataFrame


# Add lagged variables for Close and Adj_Close
dd_px['Close_lag'] = dd_px['Close'].shift(1)
#for index, value in dd_px['Stock Splits'].items():
dd_px['Adj Close'] =  dd_px['Close'] - dd_px['Dividends']
 
dd_px['Adj_Close_lag'] = dd_px['Adj Close'].shift(1)

# Calculate returns based on Adjusted Close
dd_px['returns'] = (dd_px['Adj Close'] / dd_px['Adj_Close_lag']) - 1

# Add the hi_lo_range
dd_px['hi_lo_range'] = dd_px['High'] - dd_px['Low']

# Assign the result to dd_feat
dd_feat = dd_px
dd_feat

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,year,Close_lag,Adj Close,Adj_Close_lag,returns,hi_lo_range
npartitions=558,,,,,,,,,,,,,,,
,"datetime64[ns, America/New_York]",float64,float64,float64,float64,int64,float64,float64,string,int32,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [12]:
# Write your code below.

# Convert Dask DataFrame to Pandas DataFrame

df_feat = dd_feat.compute()
# Calculate rolling average return with a window of 10 days
df_feat['rolling_avg_return'] = df_feat['returns'].rolling(window=10).mean()

In [11]:
df_feat['rolling_avg_return'] = df_feat.groupby('ticker')['returns'].rolling(window=10).mean().reset_index(level=0, drop=True)


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)


<font color='green'>**No, it wasn't necessary to to do taht. In fact, it would be better to do this directly in Dask for the following reasons:**
- Dask treats large datasets that do not fit into memory, while Pandas loads the whole dataset into memory.
- Converting largge  data to a Pandas DataFrame can lead to memory problems.
- Dask uses Lazy computation in processing the data.
- Dask speeds up computations like rolling averages by using parallel processing across multiple cores.</font> 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.